# Level 2 - Week 6 - 02 Tool Contracts

**Estimated time:** 60-90 minutes

## Learning Objectives

- Define tool input and output schemas
- Validate preconditions
- Handle failure modes


## Overview

Tools are mini-APIs.

A tool contract should define:

- name
- input schema
- output schema
- failure modes

## Underlying theory: tools are typed functions

You can think of a tool as a function with a type signature:

$$
f: X \rightarrow Y
$$

Where:

- $X$ is the input schema (valid JSON shape + constraints)
- $Y$ is the output schema

Schema validation is “type checking” for agents.

### Preconditions and postconditions

For reliability, define:

- preconditions: what must be true before executing the tool
- postconditions: what the tool guarantees on success

Example for `search`:

- preconditions: `query` non-empty, `top_k` within bounds
- postconditions: `hits` is a list, each hit has `chunk_id` and `score`

## Failure mode taxonomy

Agents should distinguish:

- invalid input (agent bug) → do not retry; fix arguments
- transient failure (timeouts, 429) → bounded retry with backoff
- permanent failure (401/403, missing index) → stop or escalate
- empty result (valid but unhelpful) → choose a different action (reformulate query, ask user)

## Practice Steps

- Define a schema for `search`.
- Validate inputs before executing.
- Validate outputs before using them in decisions.

### Sample code

Minimal tool schema and validation.


In [ ]:
tool_schema = {
    'name': 'search',
    'input_schema': {
        'query': 'string',
        'top_k': 'int',
        'filters': 'dict|none',
    },
    'output_schema': {
        'hits': 'list',
    },
}

print(tool_schema)


### Student fill-in

Implement validation for:

- preconditions (input must be valid before tool execution)
- postconditions (output must match the expected shape before the agent uses it)

Then test two failure cases:

- invalid input (should fail fast, no retries)
- malformed output (should be treated as a tool failure)

In [ ]:
def validate_search_input(query: str, top_k: int) -> None:
    if not isinstance(query, str) or not query.strip():
        raise ValueError("query must be a non-empty string")
    if not isinstance(top_k, int):
        raise ValueError("top_k must be an int")
    if top_k < 1 or top_k > 50:
        raise ValueError("top_k must be in [1, 50]")


def validate_search_output(output: dict) -> None:
    if not isinstance(output, dict):
        raise ValueError("output must be a dict")
    hits = output.get("hits")
    if not isinstance(hits, list):
        raise ValueError("output['hits'] must be a list")
    for i, h in enumerate(hits):
        if not isinstance(h, dict):
            raise ValueError(f"hit[{i}] must be a dict")
        if not isinstance(h.get("chunk_id"), str) or not h.get("chunk_id"):
            raise ValueError(f"hit[{i}].chunk_id must be a non-empty string")
        score = h.get("score")
        if not isinstance(score, (int, float)):
            raise ValueError(f"hit[{i}].score must be a number")


def search_tool(payload: dict) -> dict:
    query = payload.get("query", "")
    top_k = payload.get("top_k", 5)
    validate_search_input(query=query, top_k=top_k)

    hits = [
        {"chunk_id": "kb#001", "score": 0.82, "text": f"Hit for: {query}"},
        {"chunk_id": "kb#002", "score": 0.74, "text": "Another hit"},
    ][:top_k]
    out = {"hits": hits}
    validate_search_output(out)
    return out


print(search_tool({"query": "refund policy", "top_k": 2}))

# Uncomment to see input validation failure:
# print(search_tool({"query": "", "top_k": 2}))
# print(search_tool({"query": "ok", "top_k": 999}))

## Self-check

- Do invalid tool inputs fail fast?
- Are output schemas explicit?
